# Installing Dependencies

In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git@add-model-idefics
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × git checkout -q add-model-idefics did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q add-model-idefics did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 25.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.t

# Importing dependencies

In [ ]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
from torchvision import transforms as transforms
import torch

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Dataset

In [ ]:
# Define a class for the CocoCaptionDataset, which encapsulates the functionality for image captioning.
class CocoCaptionDataset:

    # Constructor initializes the model checkpoint and processor using Hugging Face's AutoProcessor.
    def __init__(self):
        # Define the Hugging Face model checkpoint for image captioning.
        self.model_ckpt = "HuggingFaceM4/idefics-9b-instruct"

        # Initialize the processor using the AutoProcessor with authentication token enabled.
        self.processor = AutoProcessor.from_pretrained(self.model_ckpt, use_auth_token=True)

    # Function to load and split the dataset into training and validation sets.
    def load_data(self):
        # Load the dataset using the "load_dataset" function from the datasets library.
        # The dataset is assumed to be located at the path "cat-state/mscoco-1st-caption".
        data = load_dataset("cat-state/mscoco-1st-caption")

        # Split the loaded dataset into training and testing sets.
        # The split ratio is set to allocate 0.02% of the data for testing.
        data = data["train"].train_test_split(test_size=0.0002)

        # Retrieve the training and validation sets after the split.
        train_data = data["train"]
        val_data = data["test"]

        # Return the training and validation sets for further processing or training.
        return train_data, val_data

    # Function to convert an image to RGB format if it is not already in that format.
    def img_convert_to_rgb(self, image):
        if image.mode == "RGB":
            return image
        image_rgba = image.convert("RGBA")
        background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
        alpha_composite = Image.alpha_composite(background, image_rgba)
        alpha_composite = alpha_composite.convert("RGB")
        return alpha_composite

    # Function to transform the data for model input based on defined image processing steps.
    def transform_data(self, example):
        # Retrieve image size, mean, and standard deviation from the processor.
        img_size = self.processor.image_processor.image_size
        img_mean = self.processor.image_processor.image_mean
        img_std = self.processor.image_processor.image_std

        # Define a sequence of image transformation steps using torchvision.transforms.Compose.
        img_transform = transforms.Compose([
            self.img_convert_to_rgb,
            transforms.RandomResizedCrop((img_size, img_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=img_mean, std=img_std),
        ])

        # Create prompts by combining image URL and captions in a specific format.
        prompts = []
        for i in range(len(example['caption'])):
            caption = example['caption'][i]
            prompts.append(
                [
                    example['url'][i],
                    f"Question: Explain the picture. Answer: {caption}</s>",
                ],
            )

        # Process prompts using the processor with the specified transformation and move to GPU.
        inputs = self.processor(prompts, transform=img_transform, return_tensors="pt").to("cuda")
        # Set 'labels' in inputs to be equal to 'input_ids'.
        inputs["labels"] = inputs["input_ids"]

        # Return the transformed inputs for model training or evaluation.
        return inputs

    # Function to generate training and validation datasets with the specified transformations.
    def gen_data(self):
        # Load the training and validation datasets.
        train_dataset, val_dataset = self.load_data()

        # Set the transformation function for both training and validation datasets.
        train_dataset.set_transform(self.transform_data)
        val_dataset.set_transform(self.transform_data)

        # Return the transformed training and validation datasets.
        return train_dataset, val_dataset

http://images.cocodataset.org/train2017/000000321107.jpg, Question: Explain the picture. Answer: A woman rides a bicycle on a road next to the median.

# Model Training

In [ ]:
# Import the previously defined CocoCaptionDataset class.
from coco_caption_dataset import CocoCaptionDataset

# Define a class for the ImageCaptioningModel, which encapsulates the functionality for training and pushing the model to the Hugging Face Model Hub.
class ImageCaptioningModel:

    # Constructor initializes the CocoCaptionDataset, loads data, and sets the model checkpoint.
    def __init__(self):
        # Create an instance of CocoCaptionDataset to load and preprocess data.
        cococaptiondataset = CocoCaptionDataset()

        # Generate training and validation datasets.
        self.train_data, self.val_data = cococaptiondataset.gen_data()

        # Define the Hugging Face model checkpoint for image captioning.
        self.model_ckpt = "HuggingFaceM4/idefics-9b-instruct"

    # Function to load the vision-text2text model with quantization and specific configurations.
    def load_model(self):
        # Configure quantization settings using BitsAndBytesConfig.
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            llm_int8_skip_modules=["lm_head", "embed_tokens"],
        )

        # Load the IdeficsForVisionText2Text model with the specified quantization configuration.
        model = IdeficsForVisionText2Text.from_pretrained(self.model_ckpt, quantization_config=bnb_config, device_map="auto")

        # Return the loaded model.
        return model

    # Function to create a LoRA (LoraConfig) model using the provided model and configuration.
    def create_lora_model(self, model):
        # Configure LoRA (LoraConfig) settings.
        config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["q_proj", "k_proj", "v_proj"],
            lora_dropout=0.05,
            bias="none",
        )

        # Get the Peft model with LoRA configuration.
        lora_model = get_peft_model(model, config)

        # Print trainable parameters of the LoRA model.
        lora_model.print_trainable_parameters()

        # Return the created LoRA model.
        return lora_model

    # Function to set training arguments for the Trainer.
    def set_training_args(self):
        # Configure training arguments using TrainingArguments.
        # Define training arguments for the Trainer using the TrainingArguments class.

        # Specify the output directory where the trained model and logs will be saved.
        output_dir = "idefics-mscoco-captioner"

        # Set the learning rate for the optimizer during training.
        learning_rate = 2e-4

        # Enable mixed-precision training using 16-bit floating-point (fp16).
        fp16 = True

        # Set the number of training samples processed per batch on each device.
        per_device_train_batch_size = 2

        # Set the number of evaluation samples processed per batch on each device.
        per_device_eval_batch_size = 2

        # Accumulate gradients over this many training steps before performing a backward/update pass.
        gradient_accumulation_steps = 8

        # Disable pinning data into memory for DataLoader (dataloader_pin_memory).
        dataloader_pin_memory = False

        # Limit the total number of checkpoints saved during training.
        save_total_limit = 1

        # Define the strategy for evaluation during training (can be "steps", "epoch", or "no").
        evaluation_strategy = "steps"

        # Define the strategy for saving checkpoints during training (can be "steps", "epoch", or "no").
        save_strategy = "steps"

        # Save a checkpoint every specified number of training steps.
        save_steps = 50

        # Evaluate the model every specified number of training steps.
        eval_steps = 50

        # Log metrics every specified number of training steps.
        logging_steps = 50

        # Limit the total number of training steps during training.
        max_steps = 100

        # Keep columns that are not used during training and evaluation.
        remove_unused_columns = False

        # Load the best model at the end of training based on the evaluation metric.
        load_best_model_at_end = True

        # Specify the optimizer to be used during training (paged_adamw_8bit in this case).
        optim = "paged_adamw_8bit"

        # Specify the names of the labels in the dataset.
        label_names = ["labels"]

        # Create a TrainingArguments object with the specified configurations.
        training_args = TrainingArguments(
            output_dir=output_dir,
            learning_rate=learning_rate,
            fp16=fp16,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            dataloader_pin_memory=dataloader_pin_memory,
            save_total_limit=save_total_limit,
            evaluation_strategy=evaluation_strategy,
            save_strategy=save_strategy,
            save_steps=save_steps,
            eval_steps=eval_steps,
            logging_steps=logging_steps,
            max_steps=max_steps,
            remove_unused_columns=remove_unused_columns,
            load_best_model_at_end=load_best_model_at_end,
            optim=optim,
            label_names=label_names
        )


        # Return the configured training arguments.
        return training_args

    # Function to train the model using Trainer and push the model to the Hugging Face Models Hub.
    def train_and_push_to_hub(self):
        # Load the model and create the LoRA model.
        img_cap_model = self.load_model()
        img_cap_model = self.create_lora_model(img_cap_model)

        # Configure Trainer with the loaded model, training arguments, and datasets.
        trainer = Trainer(
            model=img_cap_model,
            args=self.set_training_args(),
            train_dataset=self.train_data,
            eval_dataset=self.val_data
        )

        # Train the model using the configured Trainer.
        trainer.train()

        # Push the trained model to the Hugging Face Model Hub.
        img_cap_model.push_to_hub("idefics-mscoco-captioner", private=False)

In [ ]:
if __name__ == "__main__":
    imagecaptioningmodel = ImageCaptioningModel()
    imagecaptioningmodel.train_and_push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/processing_auto.py:214: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 19,750,912 || all params: 8,949,438,736 || trainable%: 0.22069442098698333


Step,Training Loss,Validation Loss
50,0.998700,0.628225
100,0.705800,0.620816


adapter_model.safetensors:   0%|          | 0.00/79.1M [00:00<?, ?B/s]

# Inference

In [ ]:
# Function to perform inference using a given model, processor, and prompts.

# Parameters:
# - model: The pretrained language model for text generation.
# - processor: The processor associated with the model, responsible for tokenization and decoding.
# - prompts: List of input prompts for text generation.
# - max_new_tokens: Maximum number of new tokens to generate in the output (default is 50).
def check_inference(model, processor, prompts, max_new_tokens=50):
    # Get the tokenizer from the processor.
    tokenizer = processor.tokenizer

    # Define a list of bad words to be avoided in the generated output.
    bad_words = ["<image>", "<fake_token_around_image>"]

    # Tokenize the bad words and obtain their input_ids.
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    # Define the end-of-sequence token and obtain its input_id.
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    # Tokenize the input prompts and convert them to PyTorch tensors.
    inputs = processor(prompts, return_tensors="pt").to("cuda")

    # Generate text using the model, specifying the end-of-sequence token, bad words, and maximum tokens.
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)

    # Decode the generated token IDs into text, skipping special tokens, and print the result.
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)

In [ ]:
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

model = IdeficsForVisionText2Text.from_pretrained("HuggingFaceM4/idefics-9b-instruct", quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics-9b-instruct", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/processing_auto.py:214: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from peft import PeftModel, PeftConfig
config = PeftConfig.from_pretrained("/content/idefics-mscoco-captioner")
model = PeftModel.from_pretrained(model,"/content/idefics-mscoco-captioner")

## Sample Image:

Let's run prediction with the quantized model for the image below. \\
<img src="https://cdn.pixabay.com/photo/2018/01/14/23/12/nature-3082832_1280.jpg" width="400"/>

In [ ]:
url = "https://cdn.pixabay.com/photo/2018/01/14/23/12/nature-3082832_1280.jpg"
prompts = [
    url,
    "Question: Explain the picture. Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Question: Explain the picture. Answer: A lightning strike in the distance with mountains and trees in the background.


Playground: https://huggingface.co/spaces/HuggingFaceM4/idefics_playground